# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [140]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [141]:
orders = pd.read_csv("Orders.zip")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [142]:
#Aggregate the amount_spent for unique customers
customer_id_spent = orders.groupby(["CustomerID","Country"], as_index = False)["amount_spent"].sum()
customer_id_spent.head(10)

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
5,12352,Norway,2506.04
6,12353,Bahrain,89.00
7,12354,Spain,1079.40
8,12355,Bahrain,459.40
9,12356,Portugal,2811.43


In [143]:
#distribute amount_spent per quantile range
customer_segments = [ "Low", "Moderate", "Preferred", "VIP"]
quantile_distribution = pd.qcut(customer_id_spent["amount_spent"],[.25,.50,.75,.95, 1], labels= customer_segments)
customer_id_spent["quantile_distribution"] = quantile_distribution

In [144]:
customer_id_spent.head()

,CustomerID,Country,amount_spent,quantile_distribution
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Low


In [145]:
#select VIP and Preferred customers
customer_id_spent.loc[(customer_id_spent["quantile_distribution"] == "VIP") |
                     (customer_id_spent["quantile_distribution"] == "Preferred")].head() 
                    

,CustomerID,Country,amount_spent,quantile_distribution
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
5,12352,Norway,2506.04,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [162]:
customer_id_spent[customer_id_spent.quantile_distribution == "VIP"].groupby("Country").count()

# the country with the most VIP customers is the U.K.



,CustomerID,amount_spent,quantile_distribution
Country,,,
Australia,1,1,1
Channel Islands,1,1,1
Cyprus,1,1,1
Denmark,1,1,1
EIRE,2,2,2
Finland,1,1,1
France,9,9,9
Germany,11,11,11
Japan,2,2,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [171]:
customer_id_spent[(customer_id_spent.quantile_distribution == "VIP") | 
                 (customer_id_spent.quantile_distribution == "Preferred")].groupby("Country").count()

# the country that has the most VIP and Preferred customers combined is the United Kingdom


,CustomerID,amount_spent,quantile_distribution
Country,,,
Australia,4,4,4
Austria,1,1,1
Belgium,11,11,11
Canada,1,1,1
Channel Islands,4,4,4
Cyprus,3,3,3
Denmark,3,3,3
EIRE,3,3,3
Finland,5,5,5
